In [3]:
import numpy as np
import ioh

In [4]:
ioh.ProblemClass.GRAPH.problems

{2000: 'MaxCut2000',
 2001: 'MaxCut2001',
 2002: 'MaxCut2002',
 2003: 'MaxCut2003',
 2004: 'MaxCut2004',
 2100: 'MaxCoverage2100',
 2101: 'MaxCoverage2101',
 2102: 'MaxCoverage2102',
 2103: 'MaxCoverage2103',
 2104: 'MaxCoverage2104',
 2105: 'MaxCoverage2105',
 2106: 'MaxCoverage2106',
 2107: 'MaxCoverage2107',
 2108: 'MaxCoverage2108',
 2109: 'MaxCoverage2109',
 2110: 'MaxCoverage2110',
 2111: 'MaxCoverage2111',
 2112: 'MaxCoverage2112',
 2113: 'MaxCoverage2113',
 2114: 'MaxCoverage2114',
 2115: 'MaxCoverage2115',
 2116: 'MaxCoverage2116',
 2117: 'MaxCoverage2117',
 2118: 'MaxCoverage2118',
 2119: 'MaxCoverage2119',
 2120: 'MaxCoverage2120',
 2121: 'MaxCoverage2121',
 2122: 'MaxCoverage2122',
 2123: 'MaxCoverage2123',
 2124: 'MaxCoverage2124',
 2125: 'MaxCoverage2125',
 2126: 'MaxCoverage2126',
 2127: 'MaxCoverage2127',
 2200: 'MaxInfluence2200',
 2201: 'MaxInfluence2201',
 2202: 'MaxInfluence2202',
 2203: 'MaxInfluence2203',
 2204: 'MaxInfluence2204',
 2205: 'MaxInfluence2205',
 2206

In [5]:
max_cut = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2100, problem_class=ioh.ProblemClass.GRAPH)

In [6]:
print(max_cut.meta_data)
print(max_coverage.meta_data)

<MetaData: MaxCut2000 id: 2000 iid: 1 dim: 800>
<MetaData: MaxCoverage2100 id: 2100 iid: 1 dim: 450>


In [7]:
nb_it = 10000

In [8]:
x_mcov = cut = np.random.randint(0,2,max_coverage.meta_data.n_variables)
val = max_coverage(x_mcov)
# help(max_coverage.optimum)
for ind in ioh.ProblemClass.GRAPH.problems:
    p = ioh.get_problem(ind, problem_class=ioh.ProblemClass.GRAPH)
    if np.any(p.optimum.x):
        print(p.meta_data)

# 1/2-approximation for MaxCut

Proof in Bertrand Meyer's Advanced Algorithms lecture

In [7]:
max_cut.reset()

In [8]:
cut = np.random.randint(0,2,max_cut.meta_data.n_variables)
max_cut(cut)
print(max_cut.state)

<State evaluations: 1 optimum_found: false current_best: <Solution x: [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,

# RLS

In [9]:
import numpy as np

class RandomLocalSearch:
    'Random local search algorithm'
    def __init__(self, n: int):
        self.n: int = n
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        'For each it flip 1 bit chosen uar and keep the best of both'
        
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            index_to_flip = np.random.randint(0,problem.meta_data.n_variables)
            y = x
            y[index_to_flip] = 1 - y[index_to_flip]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [10]:
max_coverage.reset()
r = RandomLocalSearch(nb_it)
r(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

10001 -213.0 -188.0


In [11]:
max_cut.reset()
r = RandomLocalSearch(nb_it)
r(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

10001 9603.0 9784.0


# (1+1) EA 

In [12]:
class EA_1p1:
    '(1+1)-EA'
    def __init__(self, n: int, p : float):
        self.n: int = n
        self.p: float = p
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            y = x.copy()
            for ind in range(len(x)):
                y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [13]:
max_coverage.reset()
ea_11 = EA_1p1(nb_it, 1/max_coverage.meta_data.n_variables)
ea_11(max_coverage)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

10001 9603.0 9784.0


In [14]:
max_cut.reset()
ea_11 = EA_1p1(nb_it, np.log(max_cut.meta_data.n_variables)/max_cut.meta_data.n_variables)
ea_11(max_cut)
print(max_cut.state)

<State evaluations: 10001 optimum_found: false current_best: <Solution x: [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1

# (1+($\lambda, \lambda$)) EA 

In [15]:
max_cut.reset()

In [16]:
import random

class EA_1plamlam:
    '(1+(lam,lam))-EA'
    def __init__(self, n: int, lam : int, p : float, c : float):
        self.n: int = n
        self.lam: int = lam
        self.p: float = p
        self.c: float = c
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        length_x = problem.meta_data.n_variables
        x = np.random.randint(0,2,length_x)
        val = problem(x)
        for _ in range(self.n):
            L = np.random.binomial(length_x, self.p)
            xp = None
            best_val = None # we allow for worse values than x
            for _ in range(self.lam):
                bits_to_flip = random.sample(range(length_x), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 1 - new_xp[i] if np.random.binomial(1, self.p) else new_xp[i]
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            y = x.copy()
            best_val = None
            for _ in range(self.lam):
                new_y = x.copy()
                for i in range(length_x):
                    if np.random.binomial(1,self.c):
                        new_y[i] = xp[i]
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            if best_val > val:
                x = y
                val = best_val

In [17]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

11001 -200.0 -199.0


In [18]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

11001 9604.0 9607.0


# First idea 
as dealing with submodular functions, we may want to select the arguments that make it grow "the most at first" --> as when adding more elements we face diminishing returns
so from a solution x -> we build lam xs turning off L bits of uar and keeping the best -> we then build lam ys by flipping on L' bits of the best xs uniformly at random and keeping the best y --> then we turn on each bit present in y and not in x

problem: solution x keeps growing?
how to stop that?

In [19]:
class OffAndOn:
    def __init__(self, n: int, lam : int, p_off : float, p_on : float, x_init_0 : bool):
        self.n: int = n
        self.p_off: int = p_off
        self.p_on: int = p_on
        self.lam: float = lam
        self.x_init_0: bool = x_init_0
        
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        if self.x_init_0:
            x = np.zeros(problem.meta_data.n_variables, dtype=int)
        else:
            x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            L = np.random.binomial(problem.meta_data.n_variables, self.p_off)
            xp = None
            best_val = None 
            for _ in range(self.lam):
                # mutation 1 : we turn off L bits 
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 0
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            L = np.random.binomial(problem.meta_data.n_variables, self.p_on)
            y = xp.copy()
            best_val = None
            for _ in range(self.lam):
                # mutation 2 : we turn on L bits
                new_y = x.copy()
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                for i in bits_to_flip:
                    new_y[i] = 1
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            # mix:
            # if bit in y and not in x then I turn it on 
            mix = x.copy()
            for i in range(problem.meta_data.n_variables):
                if y[i] and not x[i]:
                    mix[i] = 1
            val_new = problem(mix)
            if val_new > val:
                x = mix
                val = best_val
            

In [20]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, False)
offandon(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

10921 10052.0


In [21]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, True)
offandon(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

10921 415.0


In [22]:
class OffAndOn2:
    def __init__(self, n: int, lam : int, p_off : float, p_on : float, x_init_0 : bool):
        self.n: int = n
        self.p_off: int = p_off
        self.p_on: int = p_on
        self.lam: float = lam
        self.x_init_0: bool = x_init_0
        
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        if self.x_init_0:
            x = np.zeros(problem.meta_data.n_variables, dtype=int)
        else:
            x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        
        for _ in range(self.n):
            L = np.random.binomial(problem.meta_data.n_variables, self.p_off)
            xp = None
            best_val = None 
            for _ in range(self.lam):
                # mutation 1 : we turn off L bits 
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 0
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp
                
            L = np.random.binomial(problem.meta_data.n_variables, self.p_on)
            y = xp.copy()
            best_val = None
            for _ in range(self.lam):
                # mutation 2 : we turn on L bits
                new_y = x.copy()
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                for i in bits_to_flip:
                    new_y[i] = 1
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
                    
            # mix:
            # if bit in y and not in x then I turn it on 
            mix = x.copy()
            for i in range(problem.meta_data.n_variables):
                if y[i] and not x[i]:
                    mix[i] = 1
            val_new = problem(mix)
            if val_new > val:
                self.p_off *= 1/(math.log(val_new - val+1)**(1/7))
                x = mix
                val = best_val            

In [23]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
offandon = OffAndOn2(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, False)
offandon(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

10921 9990.0


In [24]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 10
offandon = OffAndOn2(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, True)
offandon(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

10921 412.0


In [25]:
max_coverage.constraints.violation

<bound method PyCapsule.violation of <ConstraintSet: <GraphConstraint 11>>>

In [26]:
max_coverage.state

<State evaluations: 10921 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Multi-objective optimization

In [13]:
import random
from typing import Callable

In [14]:
class POSS:
    
    def __init__(self, n: int, pflip: int, b: int, I: Callable):
        self.n: int = n
        self.pflip = pflip #proba flit un bit
        self.b: int = b #budget
        self.I: Callable = I #isolation function : two solutions are comparable if they have the same I value
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:

        def constraint1(s):
            nb1 = np.count_nonzero(s[0])
            if nb1 >= 2*self.b: return -100000 # -INFINI
            return s[1]
        
        def constraint2(s):
            return -np.count_nonzero(s[0])
        
        def leq(x,y): # <=
            if constraint1(x) <= constraint1(y):
                if constraint2(x) <= constraint2(y):
                    return True
                    
        def lt(x,y):  # <
            if constraint1(x) < constraint1(y):
                if constraint2(x) <= constraint2(y):
                    return True
            if constraint1(x) <= constraint1(y):
                if constraint2(x) < constraint2(y):
                    return True
        
        def bitwise_mutation(s):
            x = s[0].copy()

            for i in range(problem.meta_data.n_variables):
                flip = 1 if random.random() < self.pflip else 0
                if flip:
                    if x[i]: x[i] = 0
                    else: x[i] = 1
            
            return (x,problem(x))
        
        s0 = np.zeros(problem.meta_data.n_variables).astype(int)
        s = (s0,problem(s0))
        P = [(s[0].copy(), s[1])]
        for t in range(self.n):
            if not t%100:
                print("Iteration : ", t)
            s = P[np.random.randint(len(P))] #Select a solution from P uar            
            ns = bitwise_mutation(s)
            
            better_than_ns = [z for z in P if self.I(z[0])==self.I(ns[0]) and lt(ns,z)]

            if not better_than_ns :
                Q = [z for z in P if self.I(z)==self.I(ns) and leq(z,ns)]
                P = [z for z in P if not any(np.array_equal(z, q) for q in Q)]
                P.append((ns[0].copy(),ns[1]))

In [15]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
poss = POSS(1000, 1/dim_pb, 10, lambda x: np.count_nonzero(x[0]))
poss(max_coverage)

Iteration :  0
Iteration :  100


/home/buzetq/.local/lib/python3.8/site-packages/numpy/core/numeric.py:2457: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a1, a2 = asarray(a1), asarray(a2)


Iteration :  200
Iteration :  300
Iteration :  400
Iteration :  500
Iteration :  600
Iteration :  700
Iteration :  800
Iteration :  900


In [16]:
max_coverage.state

<State evaluations: 1001 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Tester budget max et diminuer 

# UMDA

In [88]:
class UMDA:
    
    def __init__(self, n: int, s: int):
        self.n: int = n
        self.s: int = s #population size
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        def sample(p):
            x = np.zeros(dim,int)
            for j in range(dim):
                x[j] = 1 if random.random() < p[j] else 0
            y = problem(x)
            return (x,y)

        #Initialization
        z = []
        for i in range(self.s):
            x = np.random.randint(0,2,dim)
            y = problem(x)
            z.append((x,y))
        z.sort(key=lambda x: x[1])
        P = np.array(z[:self.s//2])

        #Optimization
        p = np.zeros(dim)
        for t in range(1,self.n):
            for j in range(dim):
                count = 0
                for k in range(self.s//2):
                    if P[k,0][j] == 1: count +=1
                
                p[j] = 2*count/self.s
                if p[j] < 1/dim: p[j] = 1/dim
                if p[j] > 1 - 1/dim: p[j] = 1 - 1/dim

            for i in range(self.s):
                z[i] = sample(p)
            z.sort(key=lambda x: x[1])
            P = np.array(z[:self.s//2])

        print(P)

In [89]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
umda = UMDA(10, 10)
umda(max_cut)

[[array([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
         0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
         1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
         0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
         1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1,
         0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
         0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
         1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
         1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
         1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
         1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
         0, 1, 0, 0, 1, 1

/tmp/ipykernel_2678/2724024649.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[:self.s//2])
/tmp/ipykernel_2678/2724024649.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[:self.s//2])


In [90]:
max_cut.state

<State evaluations: 100 optimum_found: false current_best: <Solution x: [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 

Descente de gradient

In [ ]:
class Test:
    
    def __init__(self, n: int, s: int, k:int):
        self.n: int = n
        self.s: int = s #population size
        self.k: int = k
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        def sample(p):


        #Initialization
        x0 = np.zeros(dim)
        e0 = problem(x0)
        x1 = np.random.randint(0,2,dim)
        e1 = problem(x)

        x = x0
        ex = e0
        if e1 > e0:
            x = x1
            ex = e1

        L = []
        #Optimization
        for  in range(s):
            
            for j in range(s)
                y = x.copy()
                flips = np.random.randint(0,dim,k)
                for i in flips:
                    y[i] = 0 if x[i] else 1
                L.append((x)
            
            
            
            tire s échantillons avec k bits aléatoires
            on identifie celui qui fait le plus croitre la fonction
            on le garde
            on recommence 
            
            bruit ??? = mutation
            
            

In [175]:
class Test2:
    def __init__(self, n: int, s: int, p: float, b: int):
        self.n: int = n #Number of iterations
        self.s: int = s #Population size
        self.p: float = p #Proba bitflip
        self.b: float = b #Budget
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables

        def gen_budget(b,N):
            pop = []
            for i in range(N):
                x = np.zeros(dim).astype(int)
                iflips = np.random.randint(0,dim,b)
                x[iflips] = 1
                y = problem(x)
                z = (x,y)
                pop.append(z)
            return pop
        
        def mutation(z):
            x = z[0].copy()
            for i in range(dim):
                x[i] = 1 - x[i] if random.random() < self.p else x[i]
            y = problem(x)
            return (x,y)
            
        def crossover(pop,N1,N2):
            #Calcul de la distribution de probabilités 
            dist = np.zeros(dim)
            normalization = 0
            for i in range(self.s):
                dist = dist + pop[i][0] * pop[i][1]
                normalization += pop[i][1]
            dist = dist / normalization
            #Génération de la nouvelle population
            npop = []
            for t in range(N1):
                x = np.zeros(dim).astype(int)
                for i in range(dim):
                    x[i] = 1 if random.random() < dist[i] else 0
                y = problem(x)
                z = (x,y)
                npop.append(z)
            #Elitisme
            npop.sort(key=lambda x: x[1])
            npop = npop[:N2]
            return npop

        
        #Initialization
        pop = gen_budget(self.b,self.s)

        #Optimization
        for t in range(self.n):
            for i in range(self.s):
                pop[i] = mutation(pop[i])
                
            pop = crossover(pop,2*self.s,self.s)

In [176]:
import math
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test2 = Test2(1000, 10, 1/dim, 10)
test2(max_coverage)

In [177]:
max_coverage.state

<State evaluations: 30010 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [186]:
import math
max_cut.reset()
dim = max_cut.meta_data.n_variables
test2 = Test2(1000, 10, 1/dim, 400)
test2(max_cut)

In [187]:
max_cut.state

<State evaluations: 30010 optimum_found: false current_best: <Solution x: [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1